<a href="https://colab.research.google.com/github/saisagardunna/Excelr-142/blob/main/11_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install transformers datasets torch nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail","3.0.0")
article=dataset['train'][0]['article']
print("Original Artical:\n",article)

Original Artical:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office ch

In [9]:
from transformers import BartForConditionalGeneration, BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [10]:
article=dataset['test'][0]['article']
reference_summary=dataset['test'][0]['highlights']
inputs = tokenizer(article, max_length=1024, return_tensors='pt',truncation=True)
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150,min_length=30, early_stopping=True)
generated_summary=tokenizer.decode(summary_ids[0], skip_special_tokens=True,)
print("reference_summary:\n",reference_summary)
print("generated_summary:\n",generated_summary)

reference_summary:
 Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
generated_summary:
 The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.


In [11]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=65df146b61940f696d88a381cfd02146112b2c75b206a9657753f575908a8f5d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [13]:
from rouge_score import rouge_scorer
scorer=rouge_scorer.RougeScorer(['rouge1','rouge2','rouge3'],use_stemmer=True)
score=scorer.score(reference_summary,generated_summary)
print("\nROUGE sCORES:")
for key,value in score.items():
    print(f"{key}: Precision: {value.precision:.4f},Recall:{value.recall:.4f},F1 score: {value.fmeasure:.4f}")


ROUGE sCORES:
rouge1: Precision: 0.5135,Recall:0.5588,F1 score: 0.5352
rouge2: Precision: 0.3611,Recall:0.3939,F1 score: 0.3768
rouge3: Precision: 0.2286,Recall:0.2500,F1 score: 0.2388


In [19]:
pip install bert-score

In [20]:
generated_summary = ["Candidate 1 summary", "Candidate 2 summary"]  # Example generated summaries
reference_summary = ["Reference 1 summary", "Reference 2 summary"]  # Corresponding reference summaries
assert len(generated_summary) == len(reference_summary), "Different number of candidates and references"
from bert_score import score
P, R, F1 = score(generated_summary, reference_summary, lang="en", model_type="bert-base-uncased")
print("\nBERT scores")
print(f"Precision: {P.mean():.2f}, Recall: {R.mean():.2f}, F1-score: {F1.mean():.2f}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


BERT scores
Precision: 0.83, Recall: 0.83, F1-score: 0.83


In [24]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [27]:
text= """The T5 model is a transformer-based machine learning model developed by Google Research. It is trained on a diverse dataset and performs multiple NLP tasks bu [reventing them into text-to-text problems]"""
input_text= "summarize: "+ text
input_ids= tokenizer.encode(input_text,return_tensors="pt",max_length=512,truncation=True)


In [28]:
summary_ids= model.generate(input_ids, max_length=50,num_beams=5,length_penalty=0.2,early_stopping=True)
summary= tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print("summary: \n",summary)

summary: 
 the T5 model is a transformer-based machine learning model developed by google research. it is trained on a diverse dataset and performs multiple NLP tasks.
